# Test Content Export Plugin

This Jupyter notebook demonstrate how to test the content_export.py plugin. Running this notebook allows the code to be tested without deployment.

In [1]:
import os
import sys

CUR_DIR = os.path.abspath("")
ROOT_DIR = os.path.dirname(os.path.dirname(CUR_DIR))
SRC_DIR = os.path.join(ROOT_DIR, 'src')

sys.path.append(SRC_DIR)
print(f"Added {SRC_DIR} to sys.path")

Added c:\Users\franktuan\repos\healthcare-agent-orchestrator\src to sys.path


In [2]:
from config import setup_logging
from dotenv import load_dotenv

setup_logging()
load_dotenv(os.path.join(SRC_DIR, '.env'))

True

In [3]:
from app import create_app_context


app_ctx = create_app_context()

In [ ]:
from data_models.tumor_board_summary import ClinicalTrial


doc_data = {"patient_gender":"Female","patient_age":"59","medical_history":["Initial chemotherapy with carboplatin and paclitaxel, followed by pembrolizumab.","Completed 6 cycles of carboplatin, paclitaxel, and pembrolizumab.","Maintenance therapy with pembrolizumab alone.","Planned switch to KRAS-directed therapy with adagrasib due to disease progression."],"social_history":[],"cancer_type":"Stage IV non-small cell lung cancer, adenocarcinoma","ct_scan_findings":["The calculated tumor volume is 18,014 mm³.","The longest dimension of the lung mass is 919 mm."],"x_ray_findings":["Extensive thoracic and lumbar spinal fusion surgery is present.","There is a patchy opacity in the right lower lobe.","The left lung is clear.","The heart is normal in size.","No pneumothorax."],"pathology_findings":["The probability that the tumor is malignant is approximately 58.41%.","The probability that the tumor is non-malignant is approximately 41.59%."],"treatment_plan":"Start KRAS-directed treatment with adagrasib due to disease progression. Consider systemic therapy options such as docetaxel, pemetrexed, gemcitabine, or combinations if further progression occurs. Best supportive care if ECOG PS 3–4.","clinical_trials":[ClinicalTrial(title="NCT05780307", summary="The patient is eligible for this trial. The trial requires patients to have advanced/metastatic solid tumors, an ECOG performance status of 0 or 1, and to have failed previous standard treatments. The patient meets these criteria, having completed standard first-line and second-line therapies and possessing a KRAS p.G12C mutation.")]}

In [ ]:
from semantic_kernel import Kernel
from config import load_agent_config
from data_models.chat_context import ChatContext
from data_models.plugin_configuration import PluginConfiguration
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion


patient_id = "patient_4"

kernel = Kernel()
kernel.add_service(
    AzureChatCompletion(
        service_id="default",
        deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        api_version="2024-10-21"
    )
)

chat_ctx = ChatContext("test-conversation-id")
chat_ctx.patient_id = patient_id
chat_ctx.patient_data = await app_ctx.data_access.image_accessor.get_metadata_list(patient_id)

agent_config = load_agent_config("default")
summary_agent_config = [config for config in agent_config if config["name"] == "ReportCreation"][0]

plugin_config = PluginConfiguration(
    kernel=kernel,
    chat_ctx=chat_ctx,
    agent_config=summary_agent_config,
    data_access=app_ctx.data_access,
    azureml_token_provider=app_ctx.azureml_token_provider)

## Create Patient Timeline

In [ ]:
from scenarios.default.tools.patient_data import create_plugin

patient_data_plugin = create_plugin(plugin_config)
await patient_data_plugin.create_timeline(patient_id)

## Export to Word Document

In [ ]:
from scenarios.default.tools.content_export import create_plugin

content_export_plugin = create_plugin(plugin_config)
await content_export_plugin.export_to_word_doc(**doc_data)